# 01. 학습 데이터 형식 변환


In [ ]:
import json
import os


In [ ]:
def convert_bbox_to_yolo(bbox, img_width, img_height):
    x_min, y_min, x_max, y_max = bbox

    width = x_max - x_min
    height = y_max - y_min
    x_center = x_min + width / 2
    y_center = y_min + height / 2

    # 정규화
    x_center /= img_width
    y_center /= img_height
    width /= img_width
    height /= img_height

    return x_center, y_center, width, height


In [ ]:
def convert_keypoints_to_yolo(keypoints, img_width, img_height):
    yolo_kps = []
    for kp in keypoints:
        if kp is None or len(kp) < 2:
            # 좌표가 없는 경우
            yolo_kps.extend([0.0, 0.0, 0])
        else:
            x, y = kp
            x /= img_width
            y /= img_height
            # visibility 값이 없으므로 모두 2로 처리
            yolo_kps.extend([x, y, 2])
    return yolo_kps


In [ ]:
def json_to_yolo_pose(json_path, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    with open(json_path, 'r') as f:
        data = json.load(f)

    images = {img['id']: img for img in data['images']}
    annotations = data['annotations']

    yolo_labels = {}

    for ann in annotations:
        image_id = ann['image_id']
        image_info = images[image_id]
        img_w, img_h = image_info['width'], image_info['height']
        file_name = os.path.splitext(image_info['file_name'])[0] + '.txt'

        bbox = ann['bbox']
        keypoints = ann.get('keypoints', [])
        class_id = ann['category_id']

        # bbox 정규화
        x_center, y_center, width, height = convert_bbox_to_yolo(bbox, img_w, img_h)

        # keypoints 정규화
        yolo_kps = convert_keypoints_to_yolo(keypoints, img_w, img_h)

        # 전체 포맷 구성
        label_line = f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f} " + \
                     " ".join([f"{x:.6f} {y:.6f} {v}" for x, y, v in zip(yolo_kps[::3], yolo_kps[1::3], yolo_kps[2::3])])

        if file_name not in yolo_labels:
            yolo_labels[file_name] = []
        yolo_labels[file_name].append(label_line)

    # YOLO 포즈 라벨 파일 저장
    for file_name, labels in yolo_labels.items():
        label_path = os.path.join(output_dir, file_name)
        with open(label_path, 'w') as f:
            f.write("\n".join(labels))


In [ ]:
from pathlib import Path
base_path = "D:/globit/Images/Infrared/globit_nas_07_flatfish_size_label"
json_dir = 'json/'
yolo_dir = 'yolo/'

base_path, json_dir, yolo_dir = Path(base_path), Path(json_dir), Path(yolo_dir)
json_path, yolo_path = base_path / json_dir, base_path / yolo_dir

for path in Path(json_path).glob('*.json'):
    json_to_yolo_pose(path, yolo_path)


# 02. json 데이터 시각화


In [ ]:
import json
import cv2
from pathlib import Path


def draw_annotations_on_image(json_data, image_dir, save_path=None):
    image_dir = Path(image_dir)

    images = {img['id']: img for img in json_data['images']}
    annotations = json_data['annotations']

    for ann in annotations:
        image_id = ann['image_id']
        image_info = images[image_id]
        file_name = image_info['file_name']
        width, height = image_info['width'], image_info['height']

        image_path = image_dir
        img = cv2.imread(str(image_path))

        if img is None:
            print(f"❌ 이미지 로딩 실패: {image_path}")
            continue

        bbox = ann['bbox']
        x1, y1, x2, y2 = map(int, bbox)

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        keypoints = ann.get('keypoints', [])
        for i, kp in enumerate(keypoints):
            if not kp or len(kp) != 2:
                continue
            x, y = int(kp[0]), int(kp[1])
            cv2.circle(img, (x, y), 5, (255, 0, 0), -1)
            cv2.putText(img, str(i), (x + 5, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)

        cv2.imshow("Annotation Visualization", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

        if save_path:
            save_path = Path(save_path)
            save_path.mkdir(parents=True, exist_ok=True)
            cv2.imwrite(str(save_path / file_name), img)


In [ ]:
base_path = "D:/globit/Images/Infrared/globit_nas_07_flatfish_size_label/"
image_path = "image/F2_U01_C2_A1_D2025-01-24-08-25-53_0001.jpg"
label_path = "json/F2_U01_C2_A1_D2025-01-24-08-25-53_0001.json"

base_path, image_path, label_path = Path(base_path), Path(image_path), Path(label_path)
image_dir = base_path / image_path
json_dir = base_path / label_path

with open(json_dir, 'r') as f:
    json_data = json.load(f)

draw_annotations_on_image(json_data, image_dir)


# 03. 변환 데이터 시각화


In [ ]:
import cv2
import os
from pathlib import Path


In [ ]:
def draw_yolo_pose(image_path, label_path, save_path=None):
    image_path = str(image_path)

    label_path = str(label_path)
    img = cv2.imread(image_path)

    if img is None:

        print(f"이미지를 불러올 수 없습니다: {image_path}")
        return

    img_h, img_w = img.shape[:2]

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()

        class_id = int(parts[0])

        x_center, y_center, width, height = map(float, parts[1:5])
        keypoints = list(map(float, parts[5:]))

        x_center *= img_w
        y_center *= img_h
        width *= img_w
        height *= img_h

        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        print(f"YOLO BBox → 이미지 복원: ({x1}, {y1}) → ({x2}, {y2})")

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(img, f'ID: {class_id}', (x1, y1 - 10),

                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
                    )

        for i in range(0, len(keypoints), 3):
            x = int(keypoints[i] * img_w)

            y = int(keypoints[i + 1] * img_h)
            v = int(keypoints[i + 2])

            if v > 0:
                color = (0, 0, 255) if v == 1 else (255, 0, 0)
                cv2.circle(img, (x, y), 5, color, -1)

    image_path = Path(image_path)

    cv2.imshow("YOLO Pose Visualization", img)
    cv2.imwrite(f'./data/out/{image_path.name}', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if save_path:
        save_path = Path(save_path)
        save_path.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path / image_path.name), img)


In [ ]:
base_path = "D:/globit/Images/Infrared/augment2/"
image_path = "images/F2_U01_C2_A1_D2025-01-24-08-25-53_0014_horiz_contrast-10.jpg"
label_path = "labels/F2_U01_C2_A1_D2025-01-24-08-25-53_0014_horiz_contrast-10.txt"

base_path, image_path, label_path = Path(base_path), Path(image_path), Path(label_path)
draw_yolo_pose(base_path / image_path, base_path / label_path)


In [ ]:
import cv2
import os
from pathlib import Path


def draw_yolo_pose(image_path, label_path, save_path=None):
    image_path = str(image_path)

    label_path = str(label_path)
    img = cv2.imread(image_path)

    if img is None:

        print(f"이미지를 불러올 수 없습니다: {image_path}")
        return

    img_h, img_w = img.shape[:2]

    with open(label_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        parts = line.strip().split()

        class_id = int(parts[0])

        x_center, y_center, width, height = map(float, parts[1:5])
        keypoints = list(map(float, parts[5:]))

        x_center *= img_w
        y_center *= img_h
        width *= img_w
        height *= img_h

        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)

        print(f"YOLO BBox → 이미지 복원: ({x1}, {y1}) → ({x2}, {y2})")

        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        cv2.putText(img, f'ID: {class_id}', (x1, y1 - 10),

                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2
                    )

        for i in range(0, len(keypoints), 3):
            x = int(keypoints[i] * img_w)

            y = int(keypoints[i + 1] * img_h)
            v = int(keypoints[i + 2])

            if v > 0:
                color = (0, 0, 255) if v == 1 else (255, 0, 0)
                cv2.circle(img, (x, y), 5, color, -1)

    image_path = Path(image_path)

    # cv2.imshow("YOLO Pose Visualization", img)
    cv2.imwrite(f'./{image_path.name}', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    if save_path:
        save_path = Path(save_path)
        save_path.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(save_path / image_path.name), img)


image_dir = "./Fish_Size/Data/Images/infrared/augment2/images"
label_dir = "./Fish_Size/Data/Images/infrared/augment2/labels"

image_dir, label_dir = Path(image_dir), Path(label_dir)

image_paths = [path for path in image_dir.glob('*.jpg')].sort()
label_paths = [path for path in label_dir.glob('*.txt')].sort()

n = 0
for image_path, label_path in zip(image_paths, label_paths):
    if image_path.stem == label_path.stem:
        draw_yolo_pose(image_path, label_path)
    else:
        n += 1

print(n)
